In [1]:
import tableauserverclient as TSC
import pandas as pd
from io import StringIO

In [15]:
class Tableau_Server(object):
    """docstring for Tableau_Server"""
    def __init__(self, serverip, username, password, portal):
        super().__init__()
        self.server = TSC.Server(serverip)
        tableau_auth = TSC.TableauAuth(username, password, portal)
        self.server.auth.sign_in(tableau_auth)

        self.all_projects, pagination_item = self.server.projects.get()
        self.server.use_highest_version()  # make sure to use the same api version as server
        self.server.server_info.get()

        self.all_books, pagination_item = self.server.workbooks.get()
        self.all_books_names = [wb.name for wb in self.all_books]
        print('\nThere are {} workbooks on {} Tableau Server site.'.format(pagination_item.total_available, portal))

    def download_wb(self, wb):
        # download workbook given workbook object
        self.server.workbooks.download(wb.id)
    
    def download_all_views_csv(self, wb):
        # download all views csv given workbook object
        self.server.workbooks.populate_views(wb)
        views = [view for view in wb.views]
        
        for view_item in views:
            self.server.views.populate_csv(view_item)  # request view csqv
            
            # Perform byte join on the CSV data
            string = StringIO(b''.join(view_item.csv).decode("utf-8"))
            df = pd.read_csv(string, sep=",")
            # pivot view csv so its in wide format if there is "Measure Values" column
            # if not just save to csv
            if 'Measure Values' not in df.columns.values:
                df.to_csv('{}.csv'.format(view_item.name), index=False)
                continue
            df['Measure Values'] = pd.to_numeric(df['Measure Values'].str.replace('\\,|\\$|\\%', ''))
            cols = [c for c in df.columns.values if c not in ('Measure Values', 'Measure Names')]
            df = pd.pivot_table(df, values='Measure Values', columns='Measure Names', index=cols).reset_index()
            df.to_csv('{}.csv'.format(view_item.name), index=False)
            print(f"Downloaded view: {view_item.name}")

    def process_all_workbooks(self):
        for wb in self.all_books:
            print(f"Processing workbook: {wb.name}")
            self.download_wb(wb)
            self.download_all_views_csv(wb)


In [16]:
def main():
    USERNAME = "TableauAdmin"
    PASSWORD = "CapVpnCluster$20"
    portal= '890Portal'
    SERVER_IP = 'http://10.100.252.218'
    
    ts = Tableau_Server(SERVER_IP, USERNAME, PASSWORD, portal)
    
    # process all workbooks
    ts.process_all_workbooks()
    
if __name__ == '__main__':
    main()


There are 26 workbooks on 890Portal Tableau Server site.
Processing workbook: CMO_Dashboard-WIP
Processing workbook: CMO_Dashboard
Processing workbook: Consolidated Cookbook


OSError: Cannot save file into a non-existent directory: 'Rural'